# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

`functionize-notebook` has modified this notebook during execution. The following variables have been injected:

- hoursPerWindow: 12


# Preproccess

## read data

In [3]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [4]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [5]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 935
po2 935
albumin 558
hba1c 444
lymphocyte 585
height 415
urine-ketone 343
crp 57


In [6]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1209

## split patients

In [7]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [8]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,rr,saps2,sbp,scr,sodium,sofa,use_NaHCO3,uti,wbc,weight
0,13616762,20066510,31257435,True,NaN,64,NaN,NaN,NaN,NaN,...,NaN,30,NaN,NaN,NaN,3,0.0,0,NaN,130.0
0,17922352,20070680,39895564,False,NaN,47,NaN,NaN,NaN,NaN,...,NaN,26,NaN,NaN,NaN,5,0.0,0,NaN,NaN
0,11236474,20146147,34965843,True,NaN,40,NaN,NaN,NaN,NaN,...,NaN,21,NaN,NaN,NaN,3,0.0,1,NaN,NaN
0,18135181,20173681,30217596,True,NaN,54,NaN,NaN,NaN,NaN,...,NaN,24,NaN,NaN,NaN,1,0.0,0,NaN,103.5
0,12851923,20235113,35081736,False,NaN,84,NaN,NaN,NaN,NaN,...,NaN,42,NaN,NaN,NaN,5,0.0,1,NaN,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,11084536,29816115,31328473,False,NaN,25,NaN,NaN,NaN,NaN,...,NaN,17,NaN,NaN,NaN,1,0.0,0,NaN,NaN
0,11166681,29877040,38192611,True,NaN,69,NaN,NaN,NaN,NaN,...,NaN,29,NaN,NaN,NaN,4,0.0,0,NaN,NaN
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,NaN,32,NaN,NaN,NaN,3,0.0,0,NaN,47.4
0,18624988,29915993,31559071,True,NaN,82,NaN,NaN,NaN,NaN,...,NaN,63,NaN,NaN,NaN,11,0.0,0,NaN,120.3


In [9]:
splitedPatients = patients.split(splitPartCount, splitSeed)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

In [10]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

967 242
967 242
967 242
967 242
968 241


In [11]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


dataset = []
for i, (trainPatients, testPatients) in enumerate(trainTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]

    dataset.append((npTrainX, trainY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [12]:
import pickle


(TEMP_PATH / f"train-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(dataset)
)

3943190

In [13]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


datavalset = []
for i, (trainPatients, valPatients, testPatients) in enumerate(trainValTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    npValX, *_ = patientsToNumpy(
        valPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]
    valY = [p.akdPositive for p in valPatients]

    datavalset.append((npTrainX, trainY, npValX, valY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [14]:
import pickle

(TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(datavalset)
)

3808032

# LSTM

In [15]:
import pickle
from constants import TEMP_PATH

# load if not exist dataset
if "dataset" not in locals():
    dataset = pickle.loads(
        (TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )
   

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["AUC", "accuracy", "precision", "recall"])
    return model

2024-06-20 16:34:19.924516: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-20 16:34:20.720371: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npTestX, testY) in enumerate(dataset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        class_weight=weight,
        callbacks=[early_stopping],
    )

    testY = np.array(testY)
    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


2024-06-20 16:34:21.127791: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-20 16:34:21.131280: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/media/data3/users/tubh/PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an 

 1/25 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - AUC: 0.4277 - accuracy: 0.4375 - loss: 0.6994 - precision: 0.3333 - recall: 0.1250

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - AUC: 0.5358 - accuracy: 0.4914 - loss: 0.6917 - precision: 0.4802 - recall: 0.2179 - val_AUC: 0.7185 - val_accuracy: 0.6546 - val_loss: 0.6736 - val_precision: 0.6974 - val_recall: 0.5464


Epoch 2/50


...

23/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8919 - accuracy: 0.8128 - loss: 0.4096 - precision: 0.8641 - recall: 0.7640 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.8911 - accuracy: 0.8117 - loss: 0.4111 - precision: 0.8608 - recall: 0.7643 - val_AUC: 0.7212 - val_accuracy: 0.6443 - val_loss: 0.6763 - val_precision: 0.6588 - val_recall: 0.5833


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - AUC: 0.4901 - accuracy: 0.4375 - loss: 0.8168 - precision: 0.5000 - recall: 0.3889

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6864 - accuracy: 0.6277 - loss: 0.6571 - precision: 0.6945 - recall: 0.5305 


Loses: [0.5621072053909302, 0.5725816488265991, 0.5308685898780823, 0.6222260594367981, 0.6254463791847229] 0.5826459765434265 0.0363386488764845
AUCs: [0.7815126180648804, 0.7757395505905151, 0.8184054493904114, 0.7283128499984741, 0.7201749682426453] 0.7648290872573853 0.036318675411663955
Accuracies: [0.7066115736961365, 0.7190082669258118, 0.7438016533851624, 0.6694214940071106, 0.6597510576248169] 0.6997188091278076 0.031235361394039353
Precisions: [0.703125, 0.7200000286102295, 0.7479674816131592, 0.6478873491287231, 0.7127659320831299] 0.7063491582870484 0.03282616154755675
Recals: [0.7317073345184326, 0.7317073345184326, 0.7479674816131592, 0.7540983557701111, 0.5491803288459778] 0.7029321670532227 0.07738450323253775


## Static validate 

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["AUC", "accuracy", "precision", "recall"],
    )
    return model

In [19]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

if "datavalset" not in locals():
    datavalset = pickle.loads(
        (TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )

loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npValX, valY, npTestX, testY) in enumerate(datavalset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npValX = np.nan_to_num(npValX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    valY = np.array(valY)
    testY = np.array(testY)

    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_data=(npValX, valY),
        class_weight=weight,
        callbacks=[early_stopping],
    )

    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 1:01 3s/step - AUC: 0.5595 - accuracy: 0.5625 - loss: 0.6896 - precision: 0.5769 - recall: 0.8333

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.5921 - accuracy: 0.5491 - loss: 0.6882 - precision: 0.5371 - recall: 0.7675 

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - AUC: 0.5941 - accuracy: 0.5508 - loss: 0.6879 - precision: 0.5386 - recall: 0.7664 - val_AUC: 0.7689 - val_accuracy: 0.7150 - val_loss: 0.6601 - val_precision: 0.7263 - val_recall: 0.7041


Epoch 2/50


...

23/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8738 - accuracy: 0.7977 - loss: 0.4342 - precision: 0.8500 - recall: 0.7332 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.8735 - accuracy: 0.7964 - loss: 0.4355 - precision: 0.8485 - recall: 0.7316 - val_AUC: 0.7571 - val_accuracy: 0.6891 - val_loss: 0.6406 - val_precision: 0.6900 - val_recall: 0.7041


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - AUC: 0.5317 - accuracy: 0.5000 - loss: 0.7645 - precision: 0.5714 - recall: 0.4444

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7047 - accuracy: 0.6675 - loss: 0.6299 - precision: 0.7172 - recall: 0.6111 


Loses: [0.5696191191673279, 0.567305326461792, 0.5499420166015625, 0.6162207126617432, 0.6083628535270691] 0.5822900056838989 0.02554435610793763
AUCs: [0.7820249795913696, 0.7765935659408569, 0.805629551410675, 0.7300205230712891, 0.729749321937561] 0.7648035883903503 0.030136364650070677
Accuracies: [0.6900826692581177, 0.6942148804664612, 0.7355371713638306, 0.6818181872367859, 0.6846473217010498] 0.697260046005249 0.019613556089829037
Precisions: [0.7181817889213562, 0.702479362487793, 0.7706422209739685, 0.6573426723480225, 0.7090908885002136] 0.7115473866462707 0.036228798586761736
Recals: [0.642276406288147, 0.6910569071769714, 0.6829268336296082, 0.7704917788505554, 0.6393442749977112] 0.6852192401885986 0.04745384862836874
